In [238]:
from IPython.core.display import display, HTML #für eine Breitansicht des Jupyter Notebooks
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

train = pd.read_csv('/Users/patrickbaongu/Desktop/Titanic/train.csv')
test = pd.read_csv('/Users/patrickbaongu/Desktop/Titanic/test.csv')

print(train.isnull().sum())
print(test.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [226]:
def bar_chart(feature):
    
    survived = train[train['Survived'] == 1][feature].value_counts() 
    dead = train[train['Survived'] == 0][feature].value_counts() 
    df = pd.DataFrame([survived, dead])
    df.index = ['Survived', 'Dead']
    df.plot(kind = 'bar', stacked = 'True', figsize=(15,8), title = feature)
    
    
train_test_data = [train, test]
pat = '([A-Za-z]+)\.' #see doc of 'regular expressions'
for dataset in train_test_data:
    
    dataset['Title'] = dataset['Name'].str.extract(pat, expand = False)

In [227]:
sex_mapping = {'male':0, 'female':1}

drop_features = ['Name','SibSp', 'Parch', 'Ticket']

for data in train_test_data:
    
    title_mapping = {'Mr':0, 'Mrs':1/2, 'Miss': 1}
    embark_to_num = {'S':0, 'C':1/2, 'Q':1}
    cabin_to_num = {}
    family_to_num = {}
    i = 0
    j=0
    
    for title in data['Title'].unique()[3:]:
        
        title_mapping.update({title: 3})
   
    data['Title'] = data['Title'].map(title_mapping)
    data['Sex'] = data['Sex'].map(sex_mapping)
    
    data['Age'].fillna(data.groupby('Title')['Age'].transform('median'), inplace = True)
    
    data['Embarked'] = data['Embarked'].map(embark_to_num)
    data['Embarked'].fillna(data.groupby('Pclass')['Embarked'].transform('median'), inplace = True)
    
    
    data['Fare'].replace(0,np.nan)
    data['Fare'].fillna(data.groupby('Pclass')['Fare'].transform('mean'), inplace = True)
    
    data.loc[data['Fare']<= 17, 'Fare'] = 0
    data.loc[(data['Fare']> 17) & (data['Fare']<= 28), 'Fare'] = 1/3
    data.loc[(data['Fare'] > 28) & (data['Fare']<= 62), 'Fare'] = 2/3
    data.loc[data['Fare'] > 62, 'Fare'] = 1
    
    
    data['Cabin'] = data['Cabin'].str[:1]
    
    for cabin in data['Cabin'].unique():
        
        cabin_to_num.update({cabin: i*1/7})
        i+=1
    
    data['Cabin'] = data['Cabin'].map(cabin_to_num)
    data['Cabin'].fillna(data.groupby('Pclass')['Cabin'].transform('median'), inplace = True)
  

    data['Family Size'] = 1 + data['SibSp']+ data['Parch']
    for family in data['Family Size'].unique():
        
        family_to_num.update({family: j*1/8})
        j+=1
    
    data['Family Size'] = data['Family Size'].map(family_to_num)
    
    data.loc[data['Age'] <=10 , 'Age'] = 0
    data.loc[(data['Age'] > 10) & (data['Age'] <=22) , 'Age'] = 0.2
    data.loc[(data['Age'] > 22) & (data['Age'] <=34 ), 'Age'] = 0.4
    data.loc[(data['Age'] > 34) & (data['Age'] <=45) , 'Age'] = 0.6
    data.loc[(data['Age'] > 45) & (data['Age'] <=60) , 'Age'] = 0.8
    data.loc[data['Age'] > 60, 'Age'] = 1
    
    data.drop(drop_features, axis =1 ,inplace = True)
    
    

train.drop('PassengerId', axis = 1, inplace = True)
train_data = train.drop('Survived', axis = 1)
labels = train['Survived']

In [233]:
kfold = KFold(n_splits = 8, shuffle = True, random_state = 42)
models = [KNeighborsClassifier(n_neighbors = 15), DecisionTreeClassifier(), RandomForestClassifier(), GaussianNB(), SVC()]
scores = []
for model in models:
    
    cv_score = cross_val_score(estimator = model, X = train_data, y = labels, cv = kfold, n_jobs =1, scoring = 'accuracy')
    rounded = round(np.mean(cv_score),4)
    scores.append(rounded)
max_idx = np.argmax(scores)
scores

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 

[0.807, 0.7946, 0.7924, 0.744, 0.7958]

In [234]:
test_data = test.drop('PassengerId', axis =1).copy()
clf = models[max_idx].fit(train_data, labels)
y_pred = clf.predict(test_data)

In [235]:
submission = pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived':y_pred})
submission.to_csv('submission.csv', index = False)
submission = pd.read_csv('submission.csv')

In [231]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [232]:
test.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
Fare           0
Cabin          0
Embarked       0
Title          0
Family Size    0
dtype: int64

In [196]:
train.isnull().sum()

Survived       0
Pclass         0
Sex            0
Age            0
Fare           0
Embarked       0
Title          0
Family Size    0
dtype: int64

In [223]:
test.head(100)

,PassengerId,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,Family Size
0,892,3,0,0.6,0.000000,0.125,1.0,0.0,0.111111
1,893,3,1,0.8,0.000000,0.125,0.0,0.5,0.222222
2,894,2,0,1.0,0.000000,0.125,1.0,0.0,0.111111
3,895,3,0,0.4,0.000000,0.125,0.0,0.0,0.111111
4,896,3,1,0.2,0.000000,0.125,0.0,0.5,0.333333
5,897,3,0,0.2,0.000000,0.125,0.0,0.0,0.111111
6,898,3,1,0.4,0.000000,0.125,1.0,1.0,0.111111
7,899,2,0,0.4,0.666667,0.125,0.0,0.0,0.333333
8,900,3,1,0.2,0.000000,0.125,0.5,0.5,0.111111
9,901,3,0,0.2,0.333333,0.125,0.0,0.0,0.333333


In [237]:
train.head()

,Survived,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,Family Size
0,0,3,0,0.2,0.000000,0.000000,0.0,0.0,0.000
1,1,1,1,0.6,1.000000,0.142857,0.5,0.5,0.000
2,1,3,1,0.4,0.000000,0.000000,0.0,1.0,0.125
3,1,1,1,0.6,0.666667,0.142857,0.0,0.5,0.000
4,0,3,0,0.6,0.000000,0.000000,0.0,0.0,0.125


In [236]:
test.head()

,PassengerId,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,Family Size
0,892,3,0,0.6,0.0,0.0,1.0,0.0,0.000
1,893,3,1,0.8,0.0,0.0,0.0,0.5,0.125
2,894,2,0,1.0,0.0,0.0,1.0,0.0,0.000
3,895,3,0,0.4,0.0,0.0,0.0,0.0,0.000
4,896,3,1,0.2,0.0,0.0,0.0,0.5,0.250
